In [1]:
import json
import visualize
import os
import context as c
import pandas as pd

In [2]:
def load(file_name):
    with open(f"Results/raw/{file_name}", "r") as f:
        res = json.load(f)
    return res

s_debiased = load("sdb_short.json")
l_debiased = load("sdb_long.json")
s_antonym = load("ant_short_probs.json")
l_antonym = load("ant_long_probs.json")

s_raw = visualize.get_sdb_df(s_debiased, c.context_t_i)
s_ant_raw = visualize.get_ant_prob_df(s_antonym, c.context_t_i)
l_raw = visualize.get_sdb_df(l_debiased, c.context_long_t_i)
l_ant_raw = visualize.get_ant_prob_df(l_antonym, c.context_long_t_i)

AttributeError: 'Series' object has no attribute 'applymap'

# Means

In [ ]:
#s_raw.groupby(s_raw['Ethnicity']).mean().sort_values("Difference", ascending=False)
#visualize.get_sdb_means(s_raw, 'sdb_short_means.tex')

In [ ]:
#l_raw.groupby(l_raw['Ethnicity']).mean().sort_values("Difference", ascending=False)
#visualize.get_sdb_means(l_raw, 'sdb_long_means.tex')

In [ ]:
# means on word level
import pandas 
pandas.set_option('display.max_rows', 250)

In [ ]:
#s_raw.groupby(['Ethnicity', 'Biased term']).mean()

In [ ]:
#l_raw.groupby(['Ethnicity', 'Biased term']).mean()

In [ ]:
# entity level

In [ ]:
#s_raw.groupby(['Ethnicity', 'Entity']).mean()

In [ ]:
#l_raw.groupby(['Ethnicity', 'Entity']).mean()

In [ ]:
# combine with antonyms

In [ ]:
s_ant_comb = visualize.get_sdb_ant_df(s_raw, s_ant_raw)
l_ant_comb = visualize.get_sdb_ant_df(l_raw, l_ant_raw)

visualize.get_sdb_means(s_ant_comb, 'sdb_short_means.tex')
visualize.get_sdb_means(l_ant_comb, 'sdb_long_means.tex')

#s_ant_comb.loc[ant_comb['Difference'] <= 0] # 87 rows
#s_ant_comb.loc[ant_comb['Difference'] > 0] # 163 rows

# get top n changes
visualize.get_top_n_changes(s_raw, file_name='sdb_s_top_10.tex')
visualize.get_top_n_changes(l_raw, file_name='sdb_l_top_10.tex')

# TODO should I do association scores as well?
# TODO go over antonyms
# TODO add control group? Or thats only for previous part of the work?
# TODO highlight out of vocab rows?

In [ ]:
visualize.get_sdb_ant_diff(s_ant_comb, "sdb_s_ant_diff.tex")
visualize.get_sdb_ant_diff(l_ant_comb, "sdb_l_ant_diff.tex")